<a href="https://colab.research.google.com/github/jiwon5315/BusinessAnalystPortfolio/blob/main/Obesity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Dataset Overview**
- Dataset: Obesity.txt
- Purpose: Predict people's stage of diabetes
- Number of observations: 2111

- Number of predictor variables: 16
- Categorical variables (8): gender, family history, favc, caec, smoke, scc, calc, mtrans
- Quantitative variables (8): age, height, weight, fcvc, ncp, ch20, faf, tue

STEP 1: Load 'obesity' dataset and R package libraries

In [ ]:
install.packages("car")
install.packages("caret")
install.packages("class")
install.packages("tree")
install.packages("rpart")
install.packages("rpart.plot")
install.packages("randomForest")
install.packages("party")
install.packages("FNN")
install.packages("ggplot2")
library(car)
library(caret)
library(class)
library(tree)
library(rpart)
library(rpart.plot)
library(randomForest)
library(party)
library(FNN)
library(ggplot2)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘SparseM’, ‘MatrixModels’, ‘minqa’, ‘nloptr’, ‘Rcpp’, ‘RcppEigen’, ‘carData’, ‘abind’, ‘pbkrtest’, ‘quantreg’, ‘lme4’




STEP 2: Factor categorical variables

In [ ]:
obesity <- read.table('Obesity.txt',header=TRUE)
factor_variables <- c("Gender","family_history_with_overweight", "FAVC","CAEC",
                      "SMOKE","SCC","CALC","MTRANS")
for (x in factor_variables) {
  obesity[[x]] <- as.factor(obesity[[x]])
}
obesity$NObeyesdad <- factor(obesity$NObeyesdad, level=c("Insufficient_Weight","Normal_Weight",
                        "Overweight_Level_I","Overweight_Level_II","Obesity_Type_I",
                        "Obesity_Type_II","Obesity_Type_III"))

STEP 3: Rename column names and create dummy variables

In [ ]:
colnames(obesity) <- c("Gender","Age","Height","Weight","Fam_History","Diet_HighCal",
                       "Diet_Veg","Num_Meals","Snacks","Smoke","Water","Monitor",
                       "Exercise","Tech","Alcohol","Transportation","Diabetic")

STEP 4: Partition data into train and test sets (75%-25%)

In [ ]:
set.seed(50)
train_index <- createDataPartition(obesity$Diabetic,p=.75,list = F)
data_train <- obesity[ train_index,] #1586 rows
data_test <- obesity[-train_index,] #525 rows

ERROR: Error in createDataPartition(obesity$Diabetic, p = 0.75, list = F): could not find function "createDataPartition"


STEP 5: PreProcess

In [ ]:
#--Since the outcome variable is categorical, predictors were not separated
preprocess_params <- preProcess(data_train, method = c("knnImpute","center", "scale"))
train_pred <- data_train[,-17]
test_pred <- data_test[,-17]
train_outcome <- data_train$Diabetic
test_outcome <- data_test$Diabetic
trainP_normalized <- predict(preprocess_params, train_pred)
testP_normalized <- predict(preprocess_params, test_pred)
train_normalized <- data.frame(trainP_normalized,data_train$Diabetic)
test_normalized <- data.frame(testP_normalized,data_test$Diabetic)

ERROR: Error in preProcess(data_train, method = c("knnImpute", "center", "scale")): could not find function "preProcess"


STEP 6: Cross Validation

In [ ]:
ctrl=trainControl(method = "repeatedcv", number = 10, repeats=5)

STEP 7: Build Models

In [ ]:
##--Decision Tree (Accuracy: 0.5524, NIR: 0.1657, P-value: Significant)
#set.seed(5)
class.tree <- rpart(data_train.Diabetic ~., data=train_normalized, control=rpart.control(maxdepth=2),
                    method="class")

prp(class.tree,type=1,extra=1,split.font=1,varlen=-10)

###----Prune by lower cp
pruned.ct <- prune(class.tree, cp = class.tree$cptable[which.min(class.tree$cptable[,"xerror"]),"CP"])
prp(pruned.ct, type=1, extra=1, split.font=1, varlen=-10)
pruned_pred_test <- predict(pruned.ct, test_normalized[,-17], type="class")
confusionMatrix(pruned_pred_test,test_normalized$data_test.Diabetic)

###----Variable Importance plot
varImp(pruned.ct)
plot(varImp(cv.tree),main="Decision Tree Model's Variable Importance Plot")

##--KNN Model (Accuracy: 0.8324, NIR: 0.1657, P-value: Significant)
#set.seed(3)
knnModel <- train(
  data_train.Diabetic ~ .,
  data = train_normalized,
  method = "knn",
  trControl = ctrl,
  tuneGrid = data.frame(k = c(3,5,7,10))
)

knnModel$finalModel #best k=3

best_knnModel <- train(
  data_train.Diabetic ~ .,
  data = train_normalized,
  method = "knn",
  trControl = ctrl,
  tuneGrid = data.frame(k = c(3))
)

knn_pred_test <- predict(best_knnModel, test_normalized[,-17])
confusionMatrix(knn_pred_test,test_normalized$data_test.Diabetic)

###----Variable Importance plot
varImp(best_knnModel)
plot(varImp(best_knnModel),main="KNN Model's Variable Importance Plot")

##--Bagged Decision Tree (Accuracy: 0.9584905, NIR: 0.1657, P-value: Significant)
#set.seed(123)
cv.bagged.tree=train(x=trainP_normalized,y=train_outcome,
                     method='treebag',trControl=ctrl)
cv.bagged.tree

pred.bagged.tree=predict(cv.bagged.tree, testP_normalized)

cv.bagged.tree$results
conf_matrix <- confusionMatrix(data=pred.bagged.tree,test_outcome)

###----Variable Importance plot
varImp(cv.bagged.tree)
plot(varImp(cv.bagged.tree),main="Bagged Trees Model's Variable Importance Plot")

###----Misclassification Rate
# Extract elements from the confusion matrix
conf_matrix_table <- conf_matrix$table
total <- sum(conf_matrix_table)

# Calculate misclassification rate for a 7-class problem
misclassification_rate <- (total - sum(diag(conf_matrix_table))) / total

# Print misclassification rate
print(misclassification_rate)

STEP 8: Partial Dependence Plots

In [ ]:
pred.prob.insufficient <- function(object, newdata) {
  pred<-predict(object,newdata,type="prob")
  prob.insufficient=pred[,1]
  mean(prob.insufficient)
}

pred.prob.normal <- function(object, newdata) {
  pred<-predict(object,newdata,type="prob")
  prob.normal=pred[,2]
  mean(prob.normal)
}

pred.prob.overL1 <- function(object, newdata) {
  pred<-predict(object,newdata,type="prob")
  prob.overL1=pred[,3]
  mean(prob.overL1)
}

pred.prob.overL2 <- function(object, newdata) {
  pred<-predict(object,newdata,type="prob")
  prob.overL2=pred[,4]
  mean(prob.overL2)
}

pred.prob.obT1 <- function(object, newdata) {
  pred<-predict(object,newdata,type="prob")
  prob.obT1=pred[,5]
  mean(prob.obT1)
}

pred.prob.obT2 <- function(object, newdata) {
  pred<-predict(object,newdata,type="prob")
  prob.obT2=pred[,6]
  mean(prob.obT2)
}

pred.prob.obT3 <- function(object, newdata) {
  pred<-predict(object,newdata,type="prob")
  prob.obT3=pred[,7]
  mean(prob.obT3)
}

###----Weight
library(pdp)
library(ggplot2)
pdp.insuf.weight <- partial(cv.bagged.tree, pred.var = c("Weight"),
                          pred.fun = pred.prob.insufficient, plot = FALSE)
pdp.norm.weight <- partial(cv.bagged.tree, pred.var = c("Weight"),
                          pred.fun = pred.prob.normal, plot = FALSE)
pdp.overL1.weight <- partial(cv.bagged.tree, pred.var = c("Weight"),
                          pred.fun = pred.prob.overL1, plot = FALSE)
pdp.overL2.weight <- partial(cv.bagged.tree, pred.var = c("Weight"),
                             pred.fun = pred.prob.overL2, plot = FALSE)
pdp.obT1.weight <- partial(cv.bagged.tree, pred.var = c("Weight"),
                             pred.fun = pred.prob.obT1, plot = FALSE)
pdp.obT2.weight <- partial(cv.bagged.tree, pred.var = c("Weight"),
                             pred.fun = pred.prob.obT2, plot = FALSE)
pdp.obT3.weight <- partial(cv.bagged.tree, pred.var = c("Weight"),
                           pred.fun = pred.prob.obT3, plot = FALSE)

plot(pdp.insuf.weight$Weight,pdp.insuf.weight$yhat,"l",lwd=2,
     col="cyan",main="PDP Plot for Weight",xlab="Weight",ylab="Estimated probabilities",
     ylim=c(0,0.9))
lines(pdp.norm.weight$Weight,pdp.norm.weight$yhat,lwd=2,col="azure2")
lines(pdp.overL1.weight$Weight,pdp.overL1.weight$yhat,lwd=2,col="bisque")
lines(pdp.overL2.weight$Weight,pdp.overL2.weight$yhat,lwd=2,col="coral")
lines(pdp.obT1.weight$Weight,pdp.obT1.weight$yhat,lwd=2,col="brown1")
lines(pdp.obT2.weight$Weight,pdp.obT2.weight$yhat,lwd=2,col="brown3")
lines(pdp.obT3.weight$Weight,pdp.obT3.weight$yhat,lwd=2,col="brown4")
legend(2,0.9,c("Insufficient", "Normal","Over L1", "Over L2", "Obese T1", "Obese T2", "Obese T3"),
       lty=c(1,1,1),col=c("cyan","azure2","bisque","coral","brown1","brown3","brown4"))

###----Transportation
pdp.insuf.mtran <- partial(cv.bagged.tree, pred.var = c("Transportation"),
                            pred.fun = pred.prob.insufficient, plot = FALSE)
pdp.norm.mtran <- partial(cv.bagged.tree, pred.var = c("Transportation"),
                           pred.fun = pred.prob.normal, plot = FALSE)
pdp.overL1.mtran <- partial(cv.bagged.tree, pred.var = c("Transportation"),
                             pred.fun = pred.prob.overL1, plot = FALSE)
pdp.overL2.mtran <- partial(cv.bagged.tree, pred.var = c("Transportation"),
                             pred.fun = pred.prob.overL2, plot = FALSE)
pdp.obT1.mtran <- partial(cv.bagged.tree, pred.var = c("Transportation"),
                           pred.fun = pred.prob.obT1, plot = FALSE)
pdp.obT2.mtran <- partial(cv.bagged.tree, pred.var = c("Transportation"),
                           pred.fun = pred.prob.obT2, plot = FALSE)
pdp.obT3.mtran <- partial(cv.bagged.tree, pred.var = c("Transportation"),
                           pred.fun = pred.prob.obT3, plot = FALSE)

plot(pdp.insuf.mtran$Transportation,pdp.insuf.mtran$yhat,"l",lwd=2,
     col="cyan",main="PDP Plot for Transportation Mode",xlab="Weight",ylab="Estimated probabilities",
     ylim=c(0,0.17))
lines(pdp.norm.mtran$Transportation,pdp.norm.mtran$yhat,lwd=2,col="azure2")
lines(pdp.overL1.mtran$Transportation,pdp.overL1.mtran$yhat,lwd=2,col="bisque")
lines(pdp.overL2.mtran$Transportation,pdp.overL2.mtran$yhat,lwd=2,col="coral")
lines(pdp.obT1.mtran$Transportation,pdp.obT1.mtran$yhat,lwd=2,col="brown1")
lines(pdp.obT2.mtran$Transportation,pdp.obT2.mtran$yhat,lwd=2,col="brown3")
lines(pdp.obT3.mtran$Transportation,pdp.obT3.mtran$yhat,lwd=2,col="brown4")
legend(4.5,0.1,c("Insufficient", "Normal","Over L1", "Over L2", "Obese T1", "Obese T2", "Obese T3"),
       lty=c(1,1,1),col=c("cyan","azure2","bisque","coral","brown1","brown3","brown4"))

STEP 9: Interaction Plots for Normal Weight category

In [ ]:
###----Weight - Age
pdp.normal.weight_age <- partial(cv.bagged.tree, pred.var = c("Weight","Age"),
                    pred.bagged.tree = pred.prob.normal, plot = TRUE)
###----Weight - Gender
pdp.normal.weight_gender <- partial(cv.bagged.tree, pred.var = c("Weight","Gender"),
                                 pred.bagged.tree = pred.prob.normal, plot = TRUE)

pdp.normal.weight_gender
###----Transportation - Gender
pdp.normal.transportation_gender <- partial(cv.bagged.tree, pred.var = c("Transportation","Gender"),
                                    pred.bagged.tree = pred.prob.normal, plot = TRUE)
pdp.normal.transportation_gender

STEP 10: New Data - John

In [ ]:
newdata_john <- data.frame(Gender='Male',Age=31,Height=1.63,Weight=72,
                           Fam_History='yes',Diet_HighCal='yes',Diet_Veg=2,
                           Num_Meals=3,Snacks='Frequently',Smoke='yes',Water=2,
                           Monitor='no',Exercise=0,Tech=3,Alcohol='Frequently',
                           Transportation='Automobile')
###---PreProcess John's data
dataJohn_normalized <- predict(preprocess_params, newdata_john)
pred_john <- predict(cv.bagged.tree,dataJohn_normalized,type="prob")
pred_john
###--Make
knn_john <- predict(best_knnModel, newdata_john, type="prob")
knn_john
dt_john <- predict(pruned.ct, newdata_john, type="prob")
dt_john

**Conclusion**
: There is a 64% probability that John is in the Normal_Weight category, but there’s also a 32% probability that he might be Overweight_Level_I. When sharing this information with John, we can let him know that the bagged trees model came to this conclusion as an aggregated combination of other decision tree models. We can reiterate that weight, height, vegetable intake, age and gender were probably the biggest factors that impacted the results.

However, this should come as a caution since our other models contradicted this result:
Our decision tree model predicted that John has 70.5% probability of being Obesity_Type_II and the KNN Model predicted him with Obesity_Type_III.
These models are not preferred since we have a high dimensional data and 1 decision tree models are notoriously prone to overfitting.